In [1]:
from path import path
import os
rootdir = path(u'/home/pfernique/Desktop/PyClangLite')
for filepath in (rootdir/'src'/'wrapper').walkfiles('*.[hc]*'):
    os.remove(filepath.abspath())

In [2]:
import subprocess
popen = subprocess.Popen(['clang++', '-x', 'c++', '-v', '-E', '/dev/null'], stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
out, err = popen.communicate()
sysincludes = err.splitlines()
headers = []
flags = ['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
if '#include <...> search starts here:' not in sysincludes or 'End of search list.' not in sysincludes:
    warnings.warn('System includes not computed: parsing clang command output failed', Warning)
else:
    lindex = sysincludes.index('#include <...> search starts here:')
    rindex = sysincludes.index('End of search list.')
    sysincludes = sysincludes[lindex+1:rindex]
flags.extend(['-I'+str(path(sysinclude.strip()).abspath()) for sysinclude in sysincludes])
for sysinclude in sysincludes:
    sysinclude = path(sysinclude.strip())
    clanginclude = sysinclude/'clang'
    if clanginclude.exists():
        headers.extend([sysinclude.relpathto(header) for header in clanginclude.walkfiles('*.h')])
selfcontained = rootdir/'src'/'wrapper'/'clang.h'
selfcontained.write_lines(['#include <' + str(header) + '>' for header in headers])
headers  = [selfcontained]

In [3]:
from autowig import autowig
import pickle
try:
    raise
    with open('pyclanglite.pkl', 'r') as f:
        asg = pickle.load(f)
except:
    asg = autowig.AbstractSemanticGraph()
    autowig.parser.plugin = 'pyclanglite'
    autowig.parser(asg, headers, flags=flags, bootstrap=False)
    with open('pyclanglite.pkl', 'w') as f:
        pickle.dump(asg, f)

In [4]:
controller = """\
def bootstrap_controller(asg):
    node = asg['class ::std::unique_ptr']
    node.is_copyable = False
    node.is_smart_pointer = True
"""
exec controller + "    return asg"
autowig.controller['bootstrap'] = bootstrap_controller
autowig.controller.plugin = 'bootstrap'
asg = autowig.controller(asg)

In [5]:
subset = asg.nodes('::clang::tooling::buildASTFromCodeWithArgs')
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
subset += classes
subset += classes[0].subclasses(recursive=True)
subset += classes[1].subclasses(recursive=True)

In [6]:
#-Wl,--unresolved-symbols=report-all
from autowig.boost_python_generator import caching
caching(asg)

In [7]:
autowig.generator.plugin = 'boost_python'
wrappers = autowig.generator(asg, subset,
                             module=rootdir/'src'/'wrapper'/'_pyclanglite.cpp',
                             decorator=None, closure=True)
if autowig.parser == 'libclang':
    pass

In [8]:
for wrapper in wrappers:
    wrapper.write()
session = autowig.scons(rootdir, '-k', '-j6')

In [9]:
controller += '\n' + autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=1)
exec controller + "\n    return asg"
autowig.controller['bootstrap'] = bootstrap_controller

In [10]:
autowig.controller.plugin = 'bootstrap'
asg = autowig.controller(asg)
for wrapper in wrappers:
    if wrapper.on_disk:
        wrapper.write()
session = autowig.scons(rootdir, '-k', '-j6')

In [11]:
controller += '\n' + autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=1)
exec controller + "\n    return asg"
autowig.controller['bootstrap'] = bootstrap_controller

In [12]:
autowig.controller.plugin = 'bootstrap'
asg = autowig.controller(asg)
for wrapper in wrappers:
    if wrapper.on_disk:
        wrapper.write()
session = autowig.scons(rootdir, '-k', '-j6')

In [14]:
controller += '\n' + autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=1)
exec controller + "\n    return asg"
autowig.controller['bootstrap'] = bootstrap_controller

In [15]:
autowig.controller.plugin = 'bootstrap'
asg = autowig.controller(asg)
for wrapper in wrappers:
    if wrapper.on_disk:
        wrapper.write()
session = autowig.scons(rootdir, '-k', '-j6')

In [16]:
session

In [17]:
 autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=1)

'    pass'

In [20]:
print controller

def bootstrap_controller(asg):
    node = asg['class ::std::unique_ptr']
    node.is_copyable = False
    node.is_smart_pointer = True

    for method in asg['class ::llvm::ArrayRef<clang::CharSourceRange>'].methods(access='public'):
        if method.prototype == 'bool  equals(class ::llvm::ArrayRef<clang::CharSourceRange> ) const':
            method.boost_python_export = False
            break
    for constructor in asg['class ::clang::FileScopeAsmDecl'].constructors(access='public'):
        if constructor.prototype == 'FileScopeAsmDecl(class ::clang::FileScopeAsmDecl  const &)':
            constructor.boost_python_export = False
            break
    asg['class ::clang::FileScopeAsmDecl'].is_copyable = False
    for method in asg['class ::llvm::ArrayRef<clang::FixItHint>'].methods(access='public'):
        if method.prototype == 'bool  equals(class ::llvm::ArrayRef<clang::FixItHint> ) const':
            method.boost_python_export = False
            break
    for constructor in